In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()

eur_1min_spirce = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet'
df = pd.read_parquet(eur_1min_spirce)
indicator_manager = IndicatorManager()
df_with_indicators = indicator_manager.calculate_indicators(df, indicator_timeframe='5T')
# df_with_indicators.dropna(inplace=True)

# print(df_with_indicators)
df_norm = processor.normalize_simple(df=df_with_indicators)
# df_norm.to_parquet('/Users/floriankockler/Downloads/EUR_USD_5T_indics_norm2.parquet')
df_norm

In [ ]:
pair = "EUR_USD"
parquet_path = Path("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h") / f"{pair}.parquet"
df = pd.read_parquet(parquet_path)
df.shape


In [ ]:
file_5min = '/Users/floriankockler/Downloads/EUR_USD_5T_indics_norm2.parquet'
df_5min = pd.read_parquet(file_5min)
df_5min.shape



In [ ]:
eur_1min_spirce = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet'
df = pd.read_parquet(eur_1min_spirce)
indicator_manager = IndicatorManager()
df_with_indicators = indicator_manager.calculate_indicators(df, indicator_timeframe='5T')
# df_with_indicators.dropna(inplace=True)

# print(df_with_indicators)
df_norm = processor.normalize_simple(df=df_with_indicators)
df_norm.to_parquet('/Users/floriankockler/Downloads/EUR_USD_5T_indics_norm2.parquet')
df_norm.columns

# df_with_indicators = df_with_indicators['2024-11-27 08:00':'2024-11-27 09:00']
# plt.figure(figsize=(15, 6))
# plt.plot(df_with_indicators.index, df_with_indicators['macd_hist'])
# plt.title('MACD Histogram')
# plt.xlabel('Date')
# plt.ylabel('MACD Histogram Value')
# plt.grid(True)
# plt.show()



In [ ]:

# Check for inf/-inf values in each column
inf_counts = df_norm.isin([np.inf, -np.inf]).sum()
print("Number of infinite values in each column:")
print(inf_counts[inf_counts > 0])  # Only show columns with inf values


In [ ]:
df_with_indicators.isna().sum()

In [1]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/15min/EUR_USD_15T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/29nov/norm_15min/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=50_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=3_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-11-29 11:37:39 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
Dataset split sizes:
Training: 409637 samples (70.0%)
Validation: 87779 samples (15.0%)
Test: 87780 samples (15.0%)
2024-11-29 11:37:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:315 - Selected features for observation space: ['close', 'sma_20', 'sma_50', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'senkou_span_a', 'senkou_span_b', 'tenkan_sen', 'kijun_sen']
2024-11-29 11:37:39 - ForexEnv2_flat - INFO - forex_env2_flat.py:315 - Selected features for observation space: ['close', 'sma_20', 'sma_50', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di

In [ ]:
df_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet')

In [ ]:
start_time = pd.Timestamp('2024-11-27 08:00').tz_localize('UTC')
end_time = pd.Timestamp('2024-11-27 09:00').tz_localize('UTC')

chart_manager.chart(df_with_indicators, start_time, end_time)

EUR norm Hstack

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

df_norm_5min = '/Users/floriankockler/Downloads/EUR_USD_5T_indics_norm2.parquet'

df = pd.read_parquet(df_norm_5min)

dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/28nov/norm_5min/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    vec_env = DummyVecEnv([lambda: env])
    normalized_env = VecNormalize(
        vec_env,
        norm_obs=True,
        norm_reward=True,
        clip_obs=10.0,
        clip_reward=10.0,
    )
    # Add observation shape verification
    obs = normalized_env.reset()
    print(f"Verification - Observation shape after reset: {obs.shape}")
    assert obs.shape == (1, 107), f"Expected shape (1, 107), got {obs.shape}"
    
    return normalized_env


def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=100_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)
# Add verification step before training
test_obs = train_env.reset()
print(f"Pre-training verification - Observation shape: {test_obs.shape}")
model.learn(
    total_timesteps=20_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

In [ ]:
df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
train_df['bb_percent']

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

df_norm_5min = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_norm.parquet'

df = pd.read_parquet(df_norm_5min)
df.columns

In [ ]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pathlib import Path
import os, sys
output_dir= "/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/"
Path(output_dir)
# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


from data_management.preprocessor import DataPreprocessor



processor = DataPreprocessor()



eur_norm = processor.normalize_simple(df=eur)
eur_norm


In [ ]:
eur_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')